In [ ]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

# Batagelj-Zaveršnik k-core decomposition algorithm

In [ ]:
using Graphs

function batagelj_zaversnik_k_core(g::SimpleGraph)
  n_v = nv(g)
  D = zeros(Int64, n_v)
  pos = zeros(Int64, n_v)
  vert = zeros(Int64, n_v)
  md = 0
  for v in 1:n_v
    D[v] = degree(g, v)
    md = max(md, D[v])
  end

  bin = zeros(Int64, md)

  for v in 1:n_v
    bin[D[v]] += 1
  end

  start = 1
  for d in 1:md
    num = bin[d]
    bin[d] = start
    start += num
  end

  for v in 1:n_v
    pos[v] = bin[D[v]]
    vert[pos[v]] = v
    bin[D[v]] += 1
  end

  for d in md:-1:2
    bin[d] = bin[d-1]
  end
  bin[1] = 1

  for i in 1:n_v
    v = vert[i]
    for u in neighbors(g, v)
      if D[u] > D[v]
        du = D[u]
        pu = pos[u]
        pw = bin[du]
        w = vert[pw]
        if u != w
          pos[u] = pw
          pos[w] = pu
          vert[pu] = w
          vert[pw] = u
        end
        bin[du] += 1
        D[u] -= 1
      end
    end
  end

  D
end

## Create network from Batagelj-Zaveršnik k-core array

In [ ]:
using Graphs

function batagelj_zaversnik_k_core_as_network(g::SimpleGraph, k::Int64)
  indices = batagelj_zaversnik_k_core(g)
  # create a vector of vertices that are in the k-core
  induction_vertices = [i for i in 1:length(indices) if indices[i] >= k]
  # create a subgraph of the original graph with only the vertices in the k-core
  g2, vmap = induced_subgraph(g, induction_vertices)
  g2
end

## Straight forward algoritam za dekompoziciju

Pravi novu mrežu koja sadrži samo čvorove u k-core-u

In [ ]:
using Graphs

function straight_forward_k_core(g::SimpleGraph, k::Int64)
  if nv(g) == 0
    return g
  end

  # copy the graph
  g2 = SimpleGraph()
  for i in 1:nv(g)
    add_vertex!(g2)
  end
  for e in edges(g)
    add_edge!(g2, e.src, e.dst)
  end

  # ε is the list of vertices in the k-core
  ε = vertices(g2)
  # remove vertices with degree greater than k
  achieved_k_core = true
  while true
    # vertices that will be removed
    δ = [i for i in 1:nv(g2) if degree(g2, i) < k]
    # if δ is empty, k-core network is found
    if length(δ) == 0
      break
    end
    # remove vertices from δ in g2
    rem_vertices!(g2, δ)
  end

  g2
end

In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)

sf = straight_forward_k_core(g, 6)
bz = batagelj_zaversnik_k_core_as_network(g, 6)
@show sf
@show bz

In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)

g2 = batagelj_zaversnik_k_core_as_network(g, 5)

@info nv(g)
@info nv(g2)

# Simple test graphs (15-20 nodes)

Graphs are generated by a human. They are hardcoded here.

In [ ]:
using Graphs

small_graph_1 = SimpleGraph()

for i in 1:18
  add_vertex!(small_graph_1)
end

edge_pairs_1 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (2, 8),
  (4, 7),
  (4, 8),
  (5, 6),
  (6, 7),
  (7, 8),
  (7, 10),
  (7, 11),
  (7, 12),
  (7, 13),
  (7, 15),
  (8, 9),
  (8, 10),
  (8, 12),
  (9, 16),
  (12, 14),
  (14, 18),
  (16, 17),
]

for (i, j) in edge_pairs_1
  add_edge!(small_graph_1, i, j)
end

k = maximum(core_number(small_graph_1))

@show k_core(small_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_1, k))
@show vertices(straight_forward_k_core(small_graph_1, k))

Broj čvorova koji pripadaju k-core-u na osnovu Graphs.jl implementacije je isti kao i u naše dve implementacije (Batagelj-Zaveršnik i straight forward implementacija).

In [ ]:
using Graphs

small_graph_2 = SimpleGraph()

for i in 1:17
  add_vertex!(small_graph_2)
end

edge_pairs_2 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (3, 4),
  (4, 7),
  (4, 8),
  (4, 10),
  (5, 11),
  (6, 7),
  (7, 8),
  (7, 9),
  (7, 10),
  (7, 14),
  (7, 16),
  (8, 10),
  (8, 11),
  (9, 13),
  (10, 15),
  (10, 17),
  (11, 12),
]

for (i, j) in edge_pairs_2
  add_edge!(small_graph_2, i, j)
end

k = maximum(core_number(small_graph_2))

@show k_core(small_graph_2, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_2, k))
@show vertices(straight_forward_k_core(small_graph_2, k))

In [ ]:
using Graphs

small_graph_3 = SimpleGraph()

for i in 1:23
  add_vertex!(small_graph_3)
end

egde_pairs_3 = [
  (1, 7),
  (2, 7),
  (3, 7),
  (3, 8),
  (3, 12),
  (4, 5),
  (5, 9),
  (6, 7),
  (6, 12),
  (7, 12),
  (8, 12),
  (9, 13),
  (10, 11),
  (11, 12),
  (11, 15),
  (11, 16),
  (12, 13),
  (12, 16),
  (12, 17),
  (12, 18),
  (13, 14),
  (13, 18),
  (15, 19),
  (15, 20),
  (16, 21),
  (17, 21),
  (18, 22),
  (18, 23),
]

for (i, j) in egde_pairs_3
  add_edge!(small_graph_3, i, j)
end

k = maximum(core_number(small_graph_3))

@show k_core(small_graph_3, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_3, k))
@show vertices(straight_forward_k_core(small_graph_3, k))

In [ ]:
using Graphs

large_graph_1 = SimpleGraph(100, 1650)

println(k_core(large_graph_1))
println(batagelj_zaversnik_k_core(large_graph_1))

In [ ]:
using Graphs
large_graph_2 = SimpleGraph(1000, 166500)

println(k_core(large_graph_2))
println(batagelj_zaversnik_k_core(large_graph_2))

In [ ]:
rand(1:10, 5)

## Erdos-Renyi model

In [ ]:
using Random

function erdos_renyi_simple_graph(vs::Int64, es::Int64)
  S = Iterators.product(1:vs, 1:vs) |> collect |> filter(x -> x[1] != x[2])
  Random.shuffle!(S)

  G = SimpleGraph(vs)

  for (i, j) in S[1:es]
    add_edge!(G, i, j)
  end

  G
end

In [ ]:
using Graphs

erdos_renyi_graph_1 = erdos_renyi_simple_graph(10000, 165000)

k = maximum(core_number(erdos_renyi_graph_1))

@show k_core(erdos_renyi_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(erdos_renyi_graph_1, k))
@show vertices(straight_forward_k_core(erdos_renyi_graph_1, k))

## Gilbertov model

In [ ]:
using Graphs

function gilbert_simple_graph(vs::Int64, p::Float64)
  G = SimpleGraph(vs)

  for i in 1:vs-1
    for j in i+1:vs
      if rand() < p
        add_edge!(G, i, j)
      end
    end
  end
  
  G
end

In [ ]:
using Graphs

gilbert_graph_1 = gilbert_simple_graph(10000, 0.0165)

k = maximum(core_number(gilbert_graph_1))

@show k_core(gilbert_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(gilbert_graph_1, k))
@show vertices(straight_forward_k_core(gilbert_graph_1, k))

## Barabaši-Albert model

In [ ]:
using Graphs

function barabasi_albert_simple_graph(m0::Int64, p::Float64, N::Int64, m::Int64)
  G = gilbert_simple_graph(m0, p)

  Δ = []
  for i in 1:m0
    append!(Δ, repeat([i], degree(G, i)))
  end

  for i in m0+1:N
    add_vertex!(G)

    for j in 1:m
      v = rand(Δ)
      add_edge!(G, i, v)
      append!(Δ, [v])
    end

    append!(Δ, repeat([i], m))
  end

  G
end

In [ ]:
using Graphs

barabasi_albert_graph_1 = barabasi_albert_simple_graph(100, 0.1, 200, 5)

k = maximum(core_number(barabasi_albert_graph_1))

@show k_core(barabasi_albert_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(barabasi_albert_graph_1, k))
@show vertices(straight_forward_k_core(barabasi_albert_graph_1, k))

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(barabasi_albert_graph_1,
    layout=stress,
    vertexlabels = vertices(barabasi_albert_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(barabasi_albert_graph_1)],
  )
end

## Stochastic Block model

In [ ]:
using Graphs

function stochastic_block_model_simple_graph(N::Int64, q::Int64, m::Vector{Float64}, B::Matrix{Float64})
  G = gilbert_simple_graph(N, 0.005)
  g = Vector{Int64}(undef, N)

  for i in 1:N
    r = rand()
    s = 0.0
    for k in 1:q
      s += m[k]
      if r <= s
        g[i] = k
        break
      end
    end
  end

  @show g

  for i in 1:N-1
    for j in i+1:N
      if rand() <= B[g[i], g[j]]
        add_edge!(G, i, j)
      end
    end
  end

  G
end

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

stochastic_block_model_graph_1 = stochastic_block_model_simple_graph(100, 3, [0.33, 0.33, 0.34], [0.8 0.1 0.1; 0.1 0.8 0.1; 0.1 0.1 0.8])

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(stochastic_block_model_graph_1,
    layout=stress,
    vertexlabels = vertices(stochastic_block_model_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(stochastic_block_model_graph_1)]
  )
end

## Random preferential connection Core-Periphery model

In [ ]:
using Graphs

function rand_pref_conn_core_periphery(n::Int64, p::Float64, p_in::Float64, p_out::Float64)
  n_core = Int64(round(n * p))

  G = gilbert_simple_graph(n_core, p_in)

  Δ = []
  for i in 1:n_core
    append!(Δ, repeat([i], degree(G, i)))
  end
  
  for j in n_core+1:n
    add_vertex!(G)
    for i in 1:n_core
      if rand() < p_out
        v = rand(Δ)
        add_edge!(G, v, j)
        append!(Δ, [i])
      end
    end
  end

  G
end

In [ ]:
using Graphs

function rand_pref_conn_core_periphery_2(n::Int64, p::Float64, p_in::Float64, p_out::Float64)
  n_core = Int64(round(n * p))

  G = gilbert_simple_graph(n, p_out)

  for i in n_core+1:n
    for j in 1:n_core
      if rand() < p_in
        add_edge!(G, i, j)
      end
    end
  end

  G
end

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

rand_pref_conn_core_periphery_graph_1 = rand_pref_conn_core_periphery_2(100, 0.1, 0.4, 0.02)

@show core_number(rand_pref_conn_core_periphery_graph_1)

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(rand_pref_conn_core_periphery_graph_1,
    layout=stress,
    vertexlabels = vertices(rand_pref_conn_core_periphery_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(rand_pref_conn_core_periphery_graph_1)]
  )
end

# Facebook network

Nodes: 4039

Edges: 88234

Source: https://snap.stanford.edu/data/ego-Facebook.html

In [ ]:
using Graphs

all_files = readdir("facebook", join=true)

facebook_graph = SimpleGraph(4039)

for f in filter(f -> endswith(f, ".edges"), all_files)
  ego_v = parse(Int64, match(r"facebook/(\d+).edges", f).captures[1]) + 1
  for line in eachline(f)
    i, j = split(line, " ")
    i, j = parse(Int64, i) + 1, parse(Int64, j) + 1
    add_edge!(facebook_graph, i, j)
    add_edge!(facebook_graph, ego_v, i)
    add_edge!(facebook_graph, ego_v, j)
  end
end

for line in eachline("facebook/facebook_combined.txt")
  i, j = split(line, " ")
  add_edge!(facebook_graph, parse(Int64, i) + 1, parse(Int64, j) + 1)
end

In [ ]:
using Graphs

@show nv(facebook_graph)
@show ne(facebook_graph)

In [ ]:
using Graphs
using Graphs.Parallel

core_num = core_number(facebook_graph)
deg = degree(facebook_graph)
betw = betweenness_centrality(facebook_graph)
clos = closeness_centrality(facebook_graph)

In [ ]:
using JLD2

save_object("../99_project/facebook_out/facebook_degree.jld2", deg)

In [ ]:
using Statistics

# pairwise correlation using cartesian product
correlations = cor([core_num deg betw clos], dims=1)

In [ ]:
using Plots

# plot pairwise correlation as heatmap with correlation values
heatmap(["core number", "degree", "betweenness", "closeness"], ["core number", "degree", "betweenness", "closeness"], correlations, c=:blues)
savefig("facebook_pearson_correlation.png")

In [ ]:
using Plots

plot_shell_deg = scatter(core_num, deg, xlabel="core number", ylabel="degree", legend=false)
plot_shell_betw = scatter(core_num, betw, xlabel="core number", ylabel="betweenness", legend=false)
plot_shell_clos = scatter(core_num, clos, xlabel="core number", ylabel="closeness", legend=false)

plot(plot_shell_deg, plot_shell_betw, plot_shell_clos, layout=(1, 3), size=(1200, 400))

In [ ]:
using StatsBase

spearman_correlation = Iterators.product([core_num, deg, betw, clos], [core_num, deg, betw, clos]) |> collect |> m -> map(x -> corspearman(x[1], x[2]), m)

heatmap(["core number", "degree", "betweenness", "closeness"], ["core number", "degree", "betweenness", "closeness"], spearman_correlation, c=:blues)

In [ ]:
using Graphs
using Graphs.Parallel

function macroscopic_characteristics_analysis(g::SimpleGraph)
  G = SimpleGraph(nv(g))
  for e in edges(g)
    add_edge!(G, e.src, e.dst)
  end

  k = 0
  characteristics = Vector{Vector{Float64}}()

  while true
    while true
      δ = [i for i in 1:nv(G) if degree(G, i) < k]
      if length(δ) == 0
        break
      end
      rem_vertices!(G, δ)
    end

    if nv(G) == 0
      break
    end

    num_nodes = nv(G)
    num_edges = ne(G)

    graph_density = (2 * num_edges) / (num_nodes * (num_nodes - 1))

    all_connected_components = connected_components(G)
    number_of_connected_components = length(all_connected_components)
    largest_connected_component = argmax(length, all_connected_components)
    graph_of_connected_component, _vmap = induced_subgraph(G, largest_connected_component)

    percentage_of_nodes_in_gcc = length(vertices(graph_of_connected_component)) / num_nodes
    percentage_of_edges_in_gcc = length(edges(graph_of_connected_component)) / num_edges

    distances = Parallel.floyd_warshall_shortest_paths(graph_of_connected_component).dists
    sum_of_distances = sum(distances)
    smallworldness = sum_of_distances / (num_nodes * (num_nodes - 1))

    diam = diameter(graph_of_connected_component)
    clustering_coefficient = global_clustering_coefficient(graph_of_connected_component)

    push!(characteristics, [
      convert(Float64, k),
      convert(Float64, num_nodes),
      convert(Float64, num_edges),
      graph_density,
      convert(Float64, number_of_connected_components),
      percentage_of_nodes_in_gcc,
      percentage_of_edges_in_gcc,
      smallworldness,
      convert(Float64, diam),
      clustering_coefficient
    ])
    println("Done k = $k")
    k += 1
  end

  hcat(characteristics...)
end

In [ ]:
using JLD2
using Distributed
Distributed.addprocs(10)

characteristics = macroscopic_characteristics_analysis(facebook_graph)
save_object("facebook_characteristics.jld2", characteristics)

In [ ]:
ks = map(i -> convert(Int64, i), characteristics[1, :])

nums_nodes = map(i -> convert(Int64, i), characteristics[2, :])
nums_edges = map(i -> convert(Int64, i), characteristics[3, :])
graph_densities = characteristics[4, :]
nums_connected_components = map(i -> convert(Int64, i), characteristics[5, :])
percentages_of_nodes_in_gcc = characteristics[6, :]
percentages_of_edges_in_gcc = characteristics[7, :]
smallworldnesses = characteristics[8, :]
diameters = map(i -> convert(Int64, i), characteristics[9, :])
clustering_coefficients = characteristics[10, :]

In [ ]:
using Plots

plot_shell_num_nodes = scatter(ks, nums_nodes, xlabel="k", ylabel="number of nodes", legend=false)
plot_shell_num_edges = scatter(ks, nums_edges, xlabel="k", ylabel="number of edges", legend=false)
plot_shell_graph_density = scatter(ks, graph_densities, xlabel="k", ylabel="graph density", legend=false)
plot_shell_num_connected_components = scatter(ks, nums_connected_components, xlabel="k", ylabel="number of connected components", legend=false)
plot_shell_percentage_of_nodes_in_gcc = scatter(ks, percentages_of_nodes_in_gcc, xlabel="k", ylabel="percentage of nodes in gcc", legend=false)
plot_shell_percentage_of_edges_in_gcc = scatter(ks, percentages_of_edges_in_gcc, xlabel="k", ylabel="percentage of edges in gcc", legend=false)
plot_shell_smallworldness = scatter(ks, smallworldnesses, xlabel="k", ylabel="smallworldness", legend=false)
plot_shell_diameter = scatter(ks, diameters, xlabel="k", ylabel="diameter", legend=false)
plot_shell_clustering_coefficient = scatter(ks, clustering_coefficients, xlabel="k", ylabel="clustering coefficient", legend=false)

plot(plot_shell_num_nodes, plot_shell_num_edges, plot_shell_graph_density, plot_shell_num_connected_components, plot_shell_percentage_of_nodes_in_gcc, plot_shell_percentage_of_edges_in_gcc, plot_shell_smallworldness, plot_shell_diameter, plot_shell_clustering_coefficient, layout=(3, 3), size=(1200, 800))

# Deezer network

Nodes: 28281

Edges: 92752

URL: https://snap.stanford.edu/data/feather-deezer-social.html

In [ ]:
using Graphs

all_files = readdir("deezer", join=true)

deezer_graph = SimpleGraph(28281)

for line in eachline("deezer/deezer_europe_edges.csv")
  i, j = split(line, ",")
  add_edge!(deezer_graph, parse(Int64, i) + 1, parse(Int64, j) + 1)
end

In [ ]:
@show nv(deezer_graph)
@show ne(deezer_graph)

In [ ]:
using Graphs
using JLD2
# using Graphs.Parallel
# using Distributed
# Distributed.addprocs(10)

# core_num = core_number(deezer_graph)
deg = degree(deezer_graph)
save_object("../99_project/deezer_out/deezer_degree.jld2", deg)
# betw = betweenness_centrality(deezer_graph)
# clos = closeness_centrality(deezer_graph)

In [ ]:
using Statistics

# pairwise correlation using cartesian product
correlations = cor([core_num deg betw clos], dims=1)

In [ ]:
using Plots

# plot pairwise correlation as heatmap with correlation values
heatmap(["core number", "degree", "betweenness", "closeness"], ["core number", "degree", "betweenness", "closeness"], correlations, c=:blues)

In [ ]:
using Plots

plot_shell_deg = scatter(core_num, deg, xlabel="core number", ylabel="degree", legend=false)
plot_shell_betw = scatter(core_num, betw, xlabel="core number", ylabel="betweenness", legend=false)
plot_shell_clos = scatter(core_num, clos, xlabel="core number", ylabel="closeness", legend=false)

plot(plot_shell_deg, plot_shell_betw, plot_shell_clos, layout=(1, 3), size=(1200, 400))